In [1]:
import numpy as np
import os
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.layers import Dense,Activation,Flatten,Input,Dropout,GlobalAveragePooling2D
from keras.models import Model
from keras.utils import np_utils
import random
import keras
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
base_path = r"C:\\Users\Dibakar Chaudhary\Desktop\Datasets\Images\images"
dirs = os.listdir(r"C:\\Users\Dibakar Chaudhary\Desktop\Datasets\Images\images")

In [3]:
print(dirs)

['cats', 'dogs', 'horses', 'humans']


In [4]:
folder_path = ""

In [5]:
image_data = []

In [6]:
labels = []

In [7]:
label_dict = {'cats':0,'dogs':1,'horses':2,'humans':3}

In [8]:
for ix in dirs:
    path = os.path.join(base_path,ix)
    img_data = os.listdir(path)
    for im in img_data:
        img = image.load_img(os.path.join(path,im),target_size=(224,224))
        img_array = image.img_to_array(img)
        image_data.append(img_array)
        labels.append(label_dict[ix])

In [9]:
print(len(image_data), len(labels))

808 808


In [10]:
combined = list(zip(image_data, labels))
random.shuffle(combined)

image_data[:], labels[:] = zip(*combined)

In [11]:
X_train = np.array(image_data)
Y_train = np.array(labels)

print(X_train.shape,Y_train.shape)

(808, 224, 224, 3) (808,)


In [12]:
Y_train = np_utils.to_categorical(Y_train)
print(X_train.shape,Y_train.shape)

(808, 224, 224, 3) (808, 4)


In [13]:
model = keras.applications.mobilenet.MobileNet(include_top=False,weights='imagenet',input_shape=(224,224,3))

In [14]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [15]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4,activation='softmax')(d1)

model_new = Model(inputs=model.input, outputs= fc2)
model_new.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [16]:
adam = Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [21]:
for ix in range(len(model_new.layers)-4):
    print(ix, model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x0000027887661388>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x0000027887709848>
2 <keras.layers.convolutional.Conv2D object at 0x0000027887709508>
3 <keras.layers.normalization.BatchNormalization object at 0x000002788766B888>
4 <keras.layers.advanced_activations.ReLU object at 0x0000027887312C48>
5 <keras.layers.convolutional.DepthwiseConv2D object at 0x00000278C21AE4C8>
6 <keras.layers.normalization.BatchNormalization object at 0x00000278C22F0308>
7 <keras.layers.advanced_activations.ReLU object at 0x00000278C22F0108>
8 <keras.layers.convolutional.Conv2D object at 0x00000278C22FBAC8>
9 <keras.layers.normalization.BatchNormalization object at 0x00000278C23ED5C8>
10 <keras.layers.advanced_activations.ReLU object at 0x00000278C23F7188>
11 <keras.layers.convolutional.ZeroPadding2D object at 0x00000278C23F7A48>
12 <keras.layers.convolutional.DepthwiseConv2D object at 0x00000278C24609C8>
13 <keras.layers.normalization.BatchNorma

In [22]:
for ix in range(len(model_new.layers)-4):
    model_new.layers[ix].trainable = False
    
model_new.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model_new.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [24]:
hist = model_new.fit(X_train,Y_train,
                    shuffle = True,
                    batch_size = 16,
                    epochs = 20,
                    validation_split=0.20
                    )

Train on 646 samples, validate on 162 samples
Epoch 1/20
646/646 [==============================] - 50s 78ms/step - loss: 1.9025 - accuracy: 0.2786 - val_loss: 1.7364 - val_accuracy: 0.1914
Epoch 2/20
646/646 [==============================] - 50s 78ms/step - loss: 1.8423 - accuracy: 0.2895 - val_loss: 1.7364 - val_accuracy: 0.1914
Epoch 3/20
646/646 [==============================] - 53s 82ms/step - loss: 1.8659 - accuracy: 0.2848 - val_loss: 1.7364 - val_accuracy: 0.1914
Epoch 4/20
646/646 [==============================] - 53s 83ms/step - loss: 1.9209 - accuracy: 0.2523 - val_loss: 1.7364 - val_accuracy: 0.1914
Epoch 5/20
646/646 [==============================] - 51s 79ms/step - loss: 1.8509 - accuracy: 0.2632 - val_loss: 1.7364 - val_accuracy: 0.1914
Epoch 6/20
646/646 [==============================] - 51s 79ms/step - loss: 1.8934 - accuracy: 0.2632 - val_loss: 1.7364 - val_accuracy: 0.1914
Epoch 7/20
646/646 [==============================] - 51s 79ms/step - loss: 1.9062 - accur

KeyboardInterrupt: 

In [ ]:
plt.figure(0)
plt.plot(hist.history['acc'],'b')
plt.plot(hist.history['val_acc'],'g')
plt.plot(hist.history['loss'],'black')
plt.plot(hist.history['val_loss'],'red')
plt.show()